<a href="https://colab.research.google.com/github/ChenSi521/tensorflow/blob/master/new_byusr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pwd
!cat /proc/version
!lsb_release -a
!uname -a

In [ ]:
%cd /content/drive/MyDrive/models-master/research
# 必须使用这个才行 /content/drive/MyDrive/models-master/research。另外前一行的后面不能添加注释
!pwd

In [ ]:
# https://github.com/google/protobuf/releases
# !wget https://github.com/protocolbuffers/protobuf/releases/download/v3.16.0/protoc-3.16.0-linux-x86_64.zip
# !unzip protoc-3.16.0-linux-x86_64.zip -d protoc-my

In [ ]:
# !./protoc-my/bin/protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!pwd
!python -m pip install .

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
!pwd
!python object_detection/model_main_tf2.py  # 训练

In [ ]:
!/opt/bin/nvidia-smi


In [ ]:
# command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(object_detection/model_main_tf2.py, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])
!python object_detection/model_main_tf2.py --model_dir='/content/drive/MyDrive/Paperobjects/output-models1' --pipeline_config_path='/content/drive/MyDrive/models-master/research/object_detection/configs/tf2/ssd_mobilenet_v2_640x640_customer.config' --checkpoint_dir='/content/drive/MyDrive/Paperobjects/output-models1'

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

# Load pipeline config and build a detection model
config_path = '/content/drive/MyDrive/models-master/research/object_detection/configs/tf2/ssd_mobilenet_v2_640x640_customer.config'
configs = config_util.get_configs_from_pipeline_file(config_path)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
checkpoint_path = '/content/drive/MyDrive/Paperobjects/output-models1'
ckpt.restore(os.path.join(checkpoint_path, 'ckpt-45')).expect_partial()   # 要改成自己准备使用的ckpt

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

labelmap = '/content/drive/MyDrive/models-master/research/object_detection/data/custom_label_map_fortrain.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(labelmap)

IMAGE_PATH = '/content/drive/MyDrive/Paperobjects/test/IMG_2845_jpg.rf.81235ba124a502c1a398bce69d7be40e.jpg'
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=100,
            min_score_thresh=.3,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()


In [ ]:
# 下面的两步是转tflite
!python /content/drive/MyDrive/models-master/research/object_detection/export_tflite_graph_tf2.py --pipeline_config_path='/content/drive/MyDrive/Paperobjects/output-models1' --pipeline_config_path='/content/drive/MyDrive/models-master/research/object_detection/configs/tf2/ssd_mobilenet_v2_640x640_customer.config' --trained_checkpoint_dir='/content/drive/MyDrive/Paperobjects/output-models1' --output_directory='/content/drive/MyDrive/Paperobjects/output-models1/tflite-output'

In [ ]:
!tflite_convert --saved_model_dir='/content/drive/MyDrive/Paperobjects/output-models1/tflite-output/saved_model' --output_file='/content/drive/MyDrive/Paperobjects/output-models1/tflite-output/saved_model/detect.tflite' --input_shapes=1,640,640,3 --input_arrays=normalized_input_image_tensor --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' --inference_type=FLOAT --allow_custom_ops